In [1]:
import pandas as pd
import numpy as np
import math

In [12]:
# class TreeNode:
#     def __init__(self, data, attributes=None):
#         self.value = data  # value of the node
#         self.attributes = attributes if attributes is not None else {}  # attributes in dict
#         self.children = []  # list of child nodes
    
#     def __str__(self):
#         return str(self.value)

#     def add_child(self, child_node):
#         self.children.append(child_node)

class TreeNode:
    def __init__(self, label=None, attributes=None, children=None):
        self.value = label  # value of the node
        self.attributes = attributes if attributes is not None else {}  # attributes in dict
        self.children = children or []  # dict of child nodes
    
    def __str__(self):
        return str(self.value)

    def add_child(self, child_node):
        self.children.append(child_node)

def find_depth(node):
    # Base case: If the node has no children, its depth is 0
    if not node.children:
        return 0
    else:
        # Find the depth of each child node recursively
        child_depths = [find_depth(child) for child in node.children]
        # Return the maximum depth among child nodes + 1 (for the current node)
        return max(child_depths) + 1

# Example usage:
root = TreeNode("Root")
child1 = TreeNode("Child 1")
child2 = TreeNode("Child 2")
grandchild1 = TreeNode("Grandchild 1")
grandchild2 = TreeNode("Grandchild 2")

root.add_child(child1)
root.add_child(child2)
child1.add_child(grandchild1)
child2.add_child(grandchild2)

# Calculate the depth of the root node
root_depth = find_depth(root)
print("Depth of the root node:", root_depth)  # Should print 2


Depth of the root node: 2


In [7]:
print(str(root))

Root


In [43]:
def count_elements(lst):
    element_counts = {}
    for element in lst:
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1
    return element_counts

def find_most_common_label(labels):
    label_counts = count_elements(labels)
    most_common_label = max(label_counts, key=lambda k: label_counts[k])
    return most_common_label

# 示例的标签数据
Label = ["No", "No", "Yes", "Yes", "No", "Yes", "No", "No", "Yes", "Yes"]

most_common_label = find_most_common_label(Label)
print(most_common_label)


No


In [125]:
import math
import pandas as pd
import numpy as np

def count_elements(lst):
    element_counts = {}
    for element in lst:
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1
    return element_counts

#TODO: Check return a leaf node with the most common label
def find_most_common_label(labels):
    label_counts = count_elements(labels)
    most_common_label = max(label_counts, key=lambda k: label_counts[k])
    return most_common_label
            
def choose_best_attribute(S, Attributes, Label, Target_var, class_list, purity_measurement='IG'):
    print("starting choosing best attribute...")
    print("Target_var: ", Target_var)
    print("class_list: ", class_list)
    if purity_measurement == 'IG':
        print("starting use purity measurement as Information Gain")
        # Calculate information gain for all attributes and return the one with the maximum IG
        best_attribute = find_most_informative_feature(S, Label, Target_var, class_list)
    elif purity_measurement == 'majority':
        best_attribute = max(Attributes, key=lambda attr: calculate_me_gain(S, attr, Label))
    elif purity_measurement == 'gini':
        best_attribute = max(Attributes, key=lambda attr: calculate_gini_gain(S, attr, Label))
    else:
        print("Invalid purity measurement input. Only 'IG', 'majority', and 'gini' are supported. Defaulting to Information Gain.")
        best_attribute = max(Attributes, key=lambda attr: calculate_information_gain(S, attr, Label))

    return best_attribute


import math
from collections import Counter

# Calculates the entropy of the given data set for the target attr
import math


def calculate_total_entropy(data, label, class_list):
    df = pd.DataFrame(data)
    total_row = df.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = df[df[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    print("total_entropy:", total_entr)
    
    return total_entr


def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy



def calc_info_gain(feature_name, train_data, label, class_list):
    print("starting calculating {} information gain".format(feature_name))
    df = pd.DataFrame(train_data)
    print("data df in cal info gain: ", df)
    feature_value_list = df[feature_name].unique()
    print("feature_value_list: ",feature_value_list )
    # feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = df.shape[0]
    print("total_row: ", total_row)
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        print("starting feature_value={} in the list".format(feature_value))
        feature_value_data = df[df[feature_name] == feature_value] #filtering rows with that feature_value
        print("feature_value_data: ", feature_value_data)
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calculate_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

def find_most_informative_feature(train_data, label, Target_var, class_list):
    print("Target_var:", Target_var)
    print("starting finding the most informative feature")
    print("training data:",train_data )
    df = pd.DataFrame(train_data)
    print(df)
    # Get the feature columns (all columns except the label colum
    feature_columns = df.columns[df.columns != Target_var]
    # Convert feature_columns to a list if needed
    feature_list = feature_columns.tolist()
    print("feature_list: ", feature_list)
    max_info_gain = -1
    max_info_feature = None
    print("Target_var:", Target_var)
    print("class_list: ", class_list)
    
    for feature in feature_list:  #for each feature in the dataset
        print("starting calculating feaure gain")
        
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
        print("max info feature: ", max_info_feature)
            
    return max_info_feature

# def calculate_information_gain(data, attribute, target_attribute):
#     """
#     Calculate information gain for a given attribute in the dataset based on entropy.

#     Parameters:
#     - data: A list of dictionaries, where each dictionary represents a data point.
#     - attribute: The name of the attribute for which information gain is calculated (as a string).
#     - target_attribute: The name of the target attribute (the attribute to predict) (as a string).

#     Returns:
#     - information_gain: The information gain of the attribute.
#     """
#     # information_gain = 1
#     print("starting calculate information gain")
#     total_entropy = calculate_entropy(data, attribute, target_attribute)

#     print("total_entropy: ", total_entropy)
    
#     attribute_values = set(entry[attribute] for entry in data)
#     weighted_entropy = 0.0
    
#     for value in attribute_values:
#         subset = [entry for entry in data if entry[attribute] == value]
#         subset_entropy = calculate_entropy(subset, target_attribute)  # Pass attributes and target_attribute
#         weight = len(subset) / len(data)
#         weighted_entropy += weight * subset_entropy

    
#     information_gain = total_entropy - weighted_entropy
#     return information_gain



import math

def calculate_majority_error(data, attribute, target_attribute):
    """
    Calculate the Majority Error (ME) 
    for a given attribute in the dataset.
    """
    total_count = len(data)
    majority_error = 0.0

    for value in set([entry[attribute] for entry in data]):
        subset = [entry for entry in data if entry[attribute] == value]
        subset_count = len(subset)
        class_counts = {}

        for entry in subset:
            class_label = entry[target_attribute]
            if class_label in class_counts:
                class_counts[class_label] += 1
            else:
                class_counts[class_label] = 1

        majority_class_count = max(class_counts.values())
        majority_error += (subset_count / total_count) * (1.0 - (majority_class_count / subset_count))

    return majority_error

def calculate_me_gain(data, attribute, target_attribute):
    """
    Calculate the Gain using Majority Error (ME) 
    for a given attribute in the dataset.
    """

    me = calculate_majority_error(data, attribute, target_attribute)
    total_count = len(data)
    me_gain = 1.0 - me

    return me_gain

import math

def calculate_gini_index(data, attribute, target_attribute):
    """
    Calculate the Gini Index 
    for a given attribute in the dataset.
    """

    total_count = len(data)
    gini_index = 0.0

    for value in set([entry[attribute] for entry in data]):
        subset = [entry for entry in data if entry[attribute] == value]
        subset_count = len(subset)
        class_counts = {}

        for entry in subset:
            class_label = entry[target_attribute]
            if class_label in class_counts:
                class_counts[class_label] += 1
            else:
                class_counts[class_label] = 1

        gini_value = 1.0
        for class_count in class_counts.values():
            probability = class_count / subset_count
            gini_value -= probability ** 2

        gini_index += (subset_count / total_count) * gini_value

    return gini_index

def calculate_gini_gain(data, attribute, target_attribute):
    """
    Calculate the Gain using Gini Index 
    for a given attribute in the dataset.
    """

    gini = calculate_gini_index(data, attribute, target_attribute)
    total_count = len(data)
    gini_gain = 1.0 - gini

    return gini_gain

def ID3(S, Attributes, Label,Target_var, purity_measurement=None):
    #Default purity measurement for ID3
    if not purity_measurement:
        purity_measurement = 'IG'  # Default purity measurement
    print('purity_measurement:', purity_measurement)

    # Check if leaf mode with the same label
    unique_labels = set(Label)
    class_list = list(unique_labels)
    print("unique_labels: ", unique_labels)
    print("length of unique labels: ", len(unique_labels))
    print("class_list: ", class_list)
    if len(unique_labels) == 1:
        print('unique labels == 1')
        return TreeNode(label=unique_labels.pop())
    # Check if attribute is empty
    elif not Attributes:
        most_common_label = find_most_common_label(Label)
        print("attibute is empty, find most common label: ", most_common_label)
        return TreeNode(label=most_common_label)
    else:
        # Create a Root Node for tree
        root = TreeNode()
        print("starting create root node")
        # Choose the best attribute A to split S
        # Support the input purity measurement of IG, majorty and gini. Here we set default is IG.
        best_attribute = choose_best_attribute(S, Attributes, Label, Target_var, class_list,purity_measurement)
        print("best attribute: ", best_attribute)
        root.attributes = best_attribute

        # Remove the chosen attribute from the list of attributes
        remaining_attributes = [attr for attr in Attributes if attr != best_attribute]

        # Split S into subsets based on the values of the best attribute
        attribute_values = set(example[best_attribute] for example in S)

        # deal with the remaining attributes for subset Sv, according to A=V
        for value in attribute_values:
            Sv = [example for example in S if example[best_attribute] == value]
            # If Sv is empty, add leaf node with the most common value of label in S
            if not Sv:
                most_common_label = find_most_common_label(Label)
                root.children[value] = TreeNode(label=most_common_label)
            else:
                return ID3(Sv, remaining_attributes, Label, purity_measurement)

In [126]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "-", "+", "-"]
Target_var = 'Play'


root = ID3(S, Attributes, Label, Target_var, purity_measurement='IG')  # You can change purity_measurement as needed

purity_measurement: IG
unique_labels:  {'-', '+'}
length of unique labels:  2
class_list:  ['-', '+']
starting create root node
starting choosing best attribute...
Target_var:  Play
class_list:  ['-', '+']
starting use purity measurement as Information Gain
Target_var: Play
starting finding the most informative feature
training data: [{'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'Play': '-'}, {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Strong', 'Play': '-'}, {'Outlook': 'Overcast', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'Play': '+'}, {'Outlook': 'Rain', 'Temperature': 'Mild', 'Humidity': 'High', 'Wind': 'Weak', 'Play': '+'}, {'Outlook': 'Rain', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak', 'Play': '+'}, {'Outlook': 'Rain', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong', 'Play': '-'}, {'Outlook': 'Overcast', 'Temperature': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong', 'Play':

KeyError: "None of [Index(['-', '-', '+', '+', '+', '-', '+', '-', '+', '+', '+', '-', '+', '-'], dtype='object')] are in the [columns]"